# Answering Business Questions using SQL

In this project, we're going to practice using our SQL skills to answer business questions and get to know advanced stuff of SQL.

We'll Chinook database whiche is provided as a SQLite database file called chinook.db. A copy of the database schema is below:
![Chinook DB Schema](https://s3.amazonaws.com/dq-content/191/chinook-schema.svg)

First, let's load the database into Juputer Notebook and see the overview of all tables

In [ ]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

In [2]:
%%sql
SELECT
    name,
    type
FROM sqlite_master
WHERE type IN ("table","view");

Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


Let's see the structure of `genre` and `track` label 

In [3]:
%%sql
SELECT * FROM genre LIMIT 3;

Done.


genre_id,name
1,Rock
2,Jazz
3,Metal


In [5]:
%%sql
SELECT * FROM track LIMIT 3;


Done.


track_id,name,album_id,media_type_id,genre_id,composer,milliseconds,bytes,unit_price
1,For Those About To Rock (We Salute You),1,1,1,"Angus Young, Malcolm Young, Brian Johnson",343719,11170334,0.99
2,Balls to the Wall,2,2,1,None,342562,5510424,0.99
3,Fast As a Shark,3,2,1,"F. Baltes, S. Kaufman, U. Dirkscneider & W. Hoffman",230619,3990994,0.99


Write a query that returns each genre, with the number of tracks sold in the USA:
in absolute numbers
in percentages.

In [7]:
%%sql
WITH invoice_line_USA AS 
(
    SELECT * FROM invoice i
    INNER JOIN invoice_line il
    ON i.invoice_id = il.invoice_id
    WHERE i.billing_country = 'USA'
),
genre_quantity AS
(
   SELECT t.genre_id, SUM(ilu.quantity) quantity FROM track t
    INNER JOIN invoice_line_USA ilu
    ON t.track_id = ilu.track_id
    GROUP BY t.genre_id
)

/**select g.name genre, gq.quantity from genre_quantity gq
INNER JOIN genre g
ON gq.genre_id = g.genre_id
**/
select g.name genre, ROUND(CAST(gq.quantity AS FLOAT) / (SELECT SUM(quantity) FROM genre_quantity) * 100,2) percentage from genre_quantity gq
INNER JOIN genre g
ON gq.genre_id = g.genre_id
ORDER BY percentage DESC


Done.


genre,percentage
Rock,53.38
Alternative & Punk,12.37
Metal,11.8
R&B/Soul,5.04
Blues,3.43
Alternative,3.33
Latin,2.09
Pop,2.09
Hip Hop/Rap,1.9
Jazz,1.33


According to the table that shows the percentage of sales of genre above, we can see that rock is the most popular genre with a 53.8 percentage. The next popular one is alternative & Punch, which is followed by Metal accounting for 11.8 percentage.

Write a query that finds the total dollar amount of sales assigned to each sales support agent within the company. Add any extra attributes for that employee that you find are relevant to the analysis.

In [17]:
%%sql
WITH customer_total AS (
    SELECT c.customer_id, c.support_rep_id, SUM(i.total) AS total FROM invoice i
    INNER JOIN customer c
    ON i.customer_id = c.customer_id
    GROUP BY 1
),
sale_agent_total AS (
    SELECT e.employee_id, e.first_name || " " || e.last_name AS full_name, ROUND(SUM(c.total),2) total, e.title, e.birthdate, e.hire_date, e.city, e.state, e.country from customer_total c
    INNER JOIN employee e 
    ON c.support_rep_id = e.employee_id
    GROUP BY 1 ORDER BY 3


Done.


support_rep_id
3
5
4


All of the sale agents are from Calgary, Canada. The differences between the amount of sale is less than $500. Also, the youngest person made the largest sale.

In [19]:
%%sql
SELECT DISTINCT billing_country FROM invoice

Done.


billing_country
USA
Canada
France
Poland
India
Germany
Portugal
Finland
Brazil
Denmark


In [39]:
%%sql
WITH customer_per_country AS (
    SELECT country, COUNT(customer_id) no_customer FROM customer
    GROUP BY 1
),
total_sale_value AS (
    SELECT billing_country country, SUM(total) total FROM invoice
    GROUP BY 1
)
,
total_no_order AS (
    SELECT billing_country country, COUNT(invoice_id) no_order 
    FROM invoice
    GROUP BY 1
)
,
country_stat AS (
    SELECT cpc.country, cpc.no_customer, tsv.total total_sales, 
    tsv.total/cpc.no_customer avg_sale_customer, tsv.total/tno.no_order avg_order_value FROM customer_per_country cpc
    INNER JOIN total_sale_value tsv 
    on cpc.country = tsv.country
    INNER JOIN total_no_order tno
    ON cpc.country = tno.country
)
,
filter_other_country AS (
    SELECT 
        CASE
            WHEN cs.no_customer = 1 then 'Other'
            ELSE cs.country
        END AS group_country,
        SUM(no_customer) no_customer,
        SUM(total_sales) total_sales,
        SUM(avg_sale_customer) avg_sale_customer,
        SUM(avg_order_value) avg_order_value
    FROM country_stat cs
    GROUP BY group_country
    ORDER BY 3 DESC
)
SELECT group_country country, no_customer, total_sales, avg_sale_customer, avg_order_value FROM 
    (
        SELECT foc.*,
            CASE
                WHEN foc.group_country = 'Other' THEN 1
                ELSE 0
            END AS sort
        FROM filter_other_country foc
        ORDER BY sort ASC
    );



Done.


country,no_customer,total_sales,avg_sale_customer,avg_order_value
USA,13,1040.4899999999998,80.0376923076923,7.942671755725189
Canada,8,535.5900000000001,66.94875000000002,7.047236842105265
Brazil,5,427.68000000000006,85.53600000000002,7.0111475409836075
France,5,389.0699999999999,77.81399999999998,7.781399999999998
Germany,4,334.62,83.655,8.161463414634147
Czech Republic,2,273.24000000000007,136.62000000000003,9.108000000000002
United Kingdom,3,245.52,81.84,8.768571428571429
Portugal,2,185.13000000000002,92.56500000000001,6.383793103448276
India,2,183.14999999999998,91.57499999999999,8.72142857142857
Other,15,1094.9399999999998,1094.9399999999998,111.67606593406595
